# 1. Импортируем зависимости

In [1]:
import gym 
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

In [2]:
from ale_py.roms import Breakout
from ale_py import ALEInterface

ale = ALEInterface()
ale.loadROM(Breakout)

# 2. Тестируем среду 

In [3]:
environment_name = "ALE/Assault-v5"

In [4]:
env = gym.make(environment_name, render_mode="rgb_array")

In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        result = env.step(action)
        if isinstance(result, tuple) and len(result) >= 4:
            n_state, reward, done, info = result[:4]
            score += reward
        else:
            raise ValueError("The environment's step function does not return the expected number of values.")
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:210.0
Episode:2 Score:189.0
Episode:3 Score:168.0
Episode:4 Score:315.0
Episode:5 Score:273.0


In [6]:
env.action_space

Discrete(7)

In [7]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

# 3. Векторизация среды и обучение модели

In [8]:
env = make_atari_env(environment_name, n_envs=4, seed=0)

In [9]:
env = VecFrameStack(env, n_stack=4)

In [10]:
log_path = os.path.join('Training', 'Logs')

In [11]:
model = A2C("CnnPolicy", env, verbose=1, tensorboard_log=log_path)

In [12]:
model.learn(total_timesteps = 100_000)

# 4. Сохранение и перезагрузка модели

In [13]:
a2c_path = os.path.join('Training', 'Saved Models', 'A2C100_model')

In [14]:
# model.save(a2c_path)

In [15]:
# del model

In [16]:
env = make_atari_env(environment_name, n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [17]:
model = A2C.load(a2c_path, env)

Wrapping the env in a VecTransposeImage.


# 5. Оценка и тестирование

In [18]:
# env = gym.make(environment_name, render_mode="human")
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\nonam\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


(438.9, 114.42591489693233)

In [19]:
# env = make_atari_env(environment_name, n_envs=1, seed=0)
# obs, info = env.reset()
# action, _states = model.predict(obs)
# obs, reward, terminated, truncated, info = env.step(action)
# if terminated or truncated:
#     obs, info = env.reset()

In [20]:
obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = env.step(action)
    env.render("human")

KeyboardInterrupt: 

In [ ]:
env.close()